# Softmax, Cross Entropy Loss 

This model focuses on training the layer prior to the classification layer. Essentially, while training the classification layer to predict individuals from different angles, you inherently train the embedding layer before the classifier. In doing so, you figure out the embeddings the model finds important. You can take these values and find the distance (euclidean in our case) between query images and images in the gallery.

[1] https://www.semanticscholar.org/paper/Person-Re-identification-by-Descriptive-and-Hirzer-Beleznai/16c7c31a7553d99f1837fc6e88e77b5ccbb346b8?p2df

[2] https://arxiv.org/abs/1701.07717

[3] https://www.cv-foundation.org/openaccess/content_iccv_2015/papers/Zheng_Scalable_Person_Re-Identification_ICCV_2015_paper.pdf 

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
cd drive/MyDrive/person-reid/deep-person-reid

This is tested via the torchreid API provided by Kaiyang Zhou who authored numerous papers within the domain. Link here: https://github.com/KaiyangZhou/deep-person-reid

Torchreid is a python API that utilizes PyTorch and carries multiple different implementations and models for user purposes.

In [ ]:
import torchreid
import os

Register our data set with torchreid.

In [ ]:
from comp_vis_data import CvDataSet

In [ ]:
torchreid.data.register_image_dataset('cv_data', CvDataSet)

Create a DataLoader with Market1501 as the training data our procured data as the evaluation. We did some data augmentations by adding transformations of random_flip and random_crop to help the model generalize. 

In [ ]:
datamanager = torchreid.data.ImageDataManager(
    root='reid-data',
    sources='market1501',
    targets='cv_data',
    height=256,
    width=128,
    batch_size_train=32,
    batch_size_test=32,
    transforms=['random_flip', 'random_crop']
)

Path to restart training from epoch after stopping early

In [ ]:
path = os.getcwd() + '/log/test-softmax-market1501-cv-test/model/model.pth.tar-30'
path

The model is built using transfer learning using resnet50. This makes the model learn from a baseline model and will help find a local minimum quicker. We initialize the loss function to softmax as test.

In [ ]:
model = torchreid.models.build_model(
    name='resnet50',
    num_classes=datamanager.num_train_pids,
    loss='softmax'
)

In [ ]:
model = model.cuda()
optimizer = torchreid.optim.build_optimizer(
    model, optim='adam', lr=0.0003
)

In [ ]:
scheduler = torchreid.optim.build_lr_scheduler(
    optimizer,
    lr_scheduler='single_step',
    stepsize=20
)

engine = torchreid.engine.ImageSoftmaxEngine(
    datamanager, model, optimizer, scheduler=scheduler
)

In [ ]:
start_epoch = torchreid.utils.resume_from_checkpoint(
    path,
    model,
    optimizer
)

Loading checkpoint from "/content/drive/My Drive/person-reid/deep-person-reid/log/test-softmax-market1501-cv-test/model/model.pth.tar-30"
Loaded model weights
Loaded optimizer
Last epoch = 30
Last rank1 = 33.3%


We train the model for 60 epochs and evaluate on our own dataset every 10 epochs. This shows us how our model is performing as it trains.

In [ ]:
engine.run(
    save_dir='log/test-softmax-market1501-cv-test',
    max_epoch=60,
    eval_freq=10,
    print_freq=10,
    start_epoch = start_epoch
)

=> Start training
epoch: [31/60][10/404]	time 0.207 (0.267)	data 0.000 (0.047)	eta 0:53:54	loss 4.7011 (4.7336)	acc 21.8750 (16.8750)	lr 0.000030
epoch: [31/60][20/404]	time 0.207 (0.238)	data 0.000 (0.023)	eta 0:47:53	loss 4.4288 (4.7268)	acc 25.0000 (17.8125)	lr 0.000030
epoch: [31/60][30/404]	time 0.207 (0.227)	data 0.000 (0.016)	eta 0:45:50	loss 4.9772 (4.6801)	acc 9.3750 (18.2292)	lr 0.000030
epoch: [31/60][40/404]	time 0.213 (0.223)	data 0.000 (0.012)	eta 0:44:55	loss 4.3988 (4.6305)	acc 34.3750 (19.1406)	lr 0.000030
epoch: [31/60][50/404]	time 0.208 (0.220)	data 0.000 (0.009)	eta 0:44:18	loss 4.6845 (4.6074)	acc 9.3750 (18.8125)	lr 0.000030
epoch: [31/60][60/404]	time 0.208 (0.218)	data 0.000 (0.008)	eta 0:43:53	loss 4.5837 (4.6102)	acc 15.6250 (18.3333)	lr 0.000030
epoch: [31/60][70/404]	time 0.210 (0.217)	data 0.000 (0.007)	eta 0:43:35	loss 4.5118 (4.5829)	acc 18.7500 (18.7946)	lr 0.000030
epoch: [31/60][80/404]	time 0.212 (0.216)	data 0.000 (0.006)	eta 0:43:22	loss 4.1285 (4.

/content/drive/MyDrive/person-reid/deep-person-reid/torchreid/metrics/distance.py:63: UserWarning: This overload of addmm_ is deprecated:
	addmm_(Number beta, Number alpha, Tensor mat1, Tensor mat2)
Consider using one of the following signatures instead:
	addmm_(Tensor mat1, Tensor mat2, *, Number beta, Number alpha) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:882.)
  distmat.addmm_(1, -2, input1, input2.t())


Computing CMC and mAP ...
** Results **
mAP: 42.8%
CMC curve
Rank-1  : 30.8%
Rank-5  : 71.8%
Rank-10 : 76.9%
Rank-20 : 79.5%
Checkpoint saved to "log/test-softmax-market1501-cv-test/model/model.pth.tar-40"
epoch: [41/60][10/404]	time 0.219 (0.277)	data 0.000 (0.049)	eta 0:37:19	loss 3.5907 (3.8026)	acc 37.5000 (36.8750)	lr 0.000030
epoch: [41/60][20/404]	time 0.221 (0.248)	data 0.000 (0.025)	eta 0:33:21	loss 3.8814 (3.9135)	acc 43.7500 (36.0938)	lr 0.000030
epoch: [41/60][30/404]	time 0.223 (0.240)	data 0.000 (0.016)	eta 0:32:10	loss 4.0824 (3.9134)	acc 31.2500 (35.8333)	lr 0.000030
epoch: [41/60][40/404]	time 0.224 (0.235)	data 0.000 (0.012)	eta 0:31:33	loss 3.8084 (3.9294)	acc 31.2500 (35.5469)	lr 0.000030
epoch: [41/60][50/404]	time 0.221 (0.233)	data 0.000 (0.010)	eta 0:31:09	loss 4.0004 (3.9463)	acc 34.3750 (34.3750)	lr 0.000030
epoch: [41/60][60/404]	time 0.223 (0.231)	data 0.000 (0.008)	eta 0:30:54	loss 3.5772 (3.9437)	acc 43.7500 (34.6354)	lr 0.000030
epoch: [41/60][70/404]	tim

This model performed with a mAP of 42.9% and has a Rank-1 and Rank-5 of 35.9% and 66.7% which is decent for Rank-5 performance. All features extracted from this model are found within the visrank_cvision_results directory